In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.sample import sample_gen

ModuleNotFoundError: No module named 'pandas'

In [84]:
## Get Data 

df_ = pd.read_csv("data_compile_agro_climate.csv", sep=';')
df_ = df_[['idsubsegmen_repair', 'latitude', 'longitude', 'obs', 'tahun', 'bulan',
       'kdprov', 'idsubfinal', 'idkab', 'idkec', 'idsegmen', 'is_puso']]
df_['geometry'] = df_.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf_ = gpd.GeoDataFrame(df_, geometry='geometry')
print(gdf_.shape)
gdf_.head(2)

(5399, 13)


,idsubsegmen_repair,latitude,longitude,obs,tahun,bulan,kdprov,idsubfinal,idkab,idkec,idsegmen,is_puso,geometry
0,320104008A1,-6.582085,106.646608,6.0,23,9,32,320104008A1239,3201,3201040,320104008,1,POINT (106.64661 -6.58209)
1,320104008A2,-6.582958,106.646563,6.0,23,9,32,320104008A2239,3201,3201040,320104008,1,POINT (106.64656 -6.58296)


### Get Pertumbuhan Padi - Pixel Value

In [22]:
# Load Get Value From Raster

raster_path = "pertumbuhan padi/PertumbuhanPadi1Thn_Jabar_50m_2020_sd_2023.tif"  # Replace with your raster file
with rasterio.open(raster_path) as src:
    coordinates = [(geom.x, geom.y) for geom in gdf_.geometry]
    pixel_values = list(src.sample(coordinates))
    
band_names = ["Band_1", "Band_2", "Band_3", "Band_4"] 
df_bands = pd.DataFrame(pixel_values, columns=band_names)
gdf_padi_ = pd.concat([gdf_, df_bands], axis=1)
gdf_padi_.head(2)

,idsubsegmen_repair,latitude,longitude,obs,tahun,bulan,kdprov,idsubfinal,idkab,idkec,...,is_puso,geometry,Band_1,Band_2,Band_3,Band_4,Band_1,Band_2,Band_3,Band_4
0,320104008A1,-6.582085,106.646608,6.0,23,9,32,320104008A1239,3201,3201040,...,1,POINT (106.64661 -6.58209),27,20,22,14,27,20,22,14
1,320104008A2,-6.582958,106.646563,6.0,23,9,32,320104008A2239,3201,3201040,...,1,POINT (106.64656 -6.58296),28,20,20,14,28,20,20,14


In [16]:
gdf_padi_.drop(['latitude','longitude'], axis='columns', inplace=True)
gdf_padi_.to_csv("data_compile_agro_tumbuh_padi.csv")

### Get Sentinel 1 Pixel Value

In [85]:
df_bridging_ksa = pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object')
df_bridging_ksa=df_bridging_ksa.iloc[:12, 1:3]
df_bridging_ksa.loc[df_bridging_ksa.obs_in_a_year == 12, "id_per_image"] = 31
df_bridging_ksa

,obs_in_a_year,id_per_image
0,1,3
1,2,5
2,3,8
3,4,10
4,5,13
5,6,15
6,7,18
7,8,20
8,9,23
9,10,25


In [115]:
gdf_ = gdf_.merge(df_bridging_ksa, left_on='bulan', right_on='obs_in_a_year')
gdf_['year'] = "20" + gdf_['tahun'].astype("str")
gdf_['year'] = gdf_['year'].astype("int")
gdf_['index'] = 31*(gdf_.year-2020)+gdf_.id_per_image
gdf_.head(2)

,idsubsegmen_repair,latitude,longitude,obs,tahun,bulan,kdprov,idsubfinal,idkab,idkec,idsegmen,is_puso,geometry,year,obs_in_a_year,id_per_image,index
0,320104008A1,-6.582085,106.646608,6.0,23,9,32,320104008A1239,3201,3201040,320104008,1,POINT (106.64661 -6.58209),2023,9,23,116
1,320104008A2,-6.582958,106.646563,6.0,23,9,32,320104008A2239,3201,3201040,320104008,1,POINT (106.64656 -6.58296),2023,9,23,116


In [116]:
# # Load Get Value From Raster

raster_path = "sentinel 1/S1VH_Jabar_50m_2020_sd_2023.tif" 
with rasterio.open(raster_path) as src:
    coordinates = [(geom.x, geom.y) for geom in gdf_.geometry]
    pixel_values = list(src.sample(coordinates))
     
df_bands = pd.DataFrame(pixel_values)
df_bands.head(2)

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,130,130,131,132,135,139,145,152,155,149,...,135,129,138,147,151,154,155,156,156,155
1,144,142,145,149,155,158,156,155,154,152,...,140,141,143,146,150,156,159,162,163,161


In [117]:
vh_list = [f"VH_{i}" for i in range(61, -1, -1)]
df_VH = pd.DataFrame(index=gdf_.index, columns = vh_list)
print(df_VH.shape)
df_VH.head(2)

(5399, 62)


,VH_61,VH_60,VH_59,VH_58,VH_57,VH_56,VH_55,VH_54,VH_53,VH_52,...,VH_9,VH_8,VH_7,VH_6,VH_5,VH_4,VH_3,VH_2,VH_1,VH_0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
for i,r in gdf_.iterrows():
    ind = r['index']
    df_VH.iloc[i,:] = df_bands.iloc[i,ind-62:ind]
    # break

In [17]:
gdf_fin_ = gdf_[['idsubsegmen_repair', 'latitude', 'longitude', 'obs', 'tahun', 'bulan',
       'kdprov', 'idsubfinal', 'idkab', 'idkec', 'idsegmen', 'is_puso', 'geometry',
        'id_per_image', 'index']].join(df_VH)

NameError: name 'gdf_' is not defined

In [14]:
gdf_fin_.drop(['latitude','longitude'], axis='columns', inplace=True)
gdf_fin_.to_csv("data_compile_agro_sentinel_1.csv")